In [495]:
import numpy as np
import pandas as pd
import torch

In [496]:
data = pd.read_csv('/content/mnist_train.csv')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data_train = data[0:42000]
data_valid = data[42000:60000]

scaler.fit(data_train)
scaler.transform(data_train)

array([[ 0.19089655,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.53801513,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.15488579,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.84645046,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.88246122,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.53801513,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [497]:
scaler.transform(data_valid)

array([[-1.1922328 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.84645046,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.1922328 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.19089655,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.53667888,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.22824356,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [498]:
from torch.utils import data
from torch.utils.data import TensorDataset, DataLoader

class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.X = X.values / X.values.mean()
        self.X = torch.tensor(self.X, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [499]:
X_train = data_train.drop(columns='label')
y_train = data_train['label']
X_valid = data_valid.drop(columns='label')
y_valid = data_valid['label']

In [500]:
import torchvision.transforms as tfs
data_tfs = tfs.Compose([
    tfs.ToTensor(),
    tfs.Normalize((0.5), (0.5))
])


train_dataloader = DataLoader(Dataset(X_train, y_train),
                                batch_size=4,
                                shuffle=True, 
                                num_workers=2,)
valid_dataloader = DataLoader(Dataset(X_valid, y_valid), 
                                batch_size=4,
                                shuffle=True, 
                                num_workers=2)

In [501]:
loaders = {"train": train_dataloader, "valid": valid_dataloader}

In [502]:
import torch
from torch import nn
from torch.nn import functional as F

features = 784

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.type(torch.float32)
        x = self.fc1(x)
        x = F.elu(x)
        x = self.fc2(x)
        x = F.elu(x)
        x = self.fc3(x)
                
        return x

net = Net()
net

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [503]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)

In [504]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [505]:
max_epochs = 10
accuracy = {'train': [], 'valid': []}
for epoch in range(max_epochs):
    for k, dataloader in loaders.items():
        epoch_correct = 0
        epoch_all = 0
        for x_batch, y_batch in dataloader:
            if k == 'train':
                net.train()
                output = net.forward(x_batch)
                y_batch = y_batch.type(torch.LongTensor)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            else:
                net.eval()
                with torch.no_grad():
                    output = net.forward(x_batch)
            preds = output.argmax(-1)
            correct =torch.tensor(y_batch[y_batch==preds].size()) 
            all = 4
            epoch_correct += correct.item()
            epoch_all += all
        if k == 'train':
            print(f"Epoch: {epoch+1}")
        print(f"Loader: {k}. Accuracy: {epoch_correct/epoch_all}")
        accuracy[k].append(epoch_correct/epoch_all)

KeyboardInterrupt: ignored

In [ ]:
x = torch.from_numpy(X_train.values)
x = torch.tensor(x, dtype=torch.float32)
y = torch.from_numpy(y_train.values)
y = torch.tensor(y, dtype=torch.float32)

In [ ]:
net.train()
x = torch.tensor(X_train[0:10].values, dtype=torch.float32)
output = net(x)
preds = output.argmax(-1)

In [ ]:
preds, y_train[0:10].values

In [ ]:
!pip install skorch

In [ ]:

train_sliceable = SliceDataset(x)
scores = cross_val_score(Net, train_sliceable,y_train, cv = 5, scoring = "accuracy")

In [509]:
class ConvNet(nn.Module):
    def __init__(self,dropout_rate=0.4,dropout_rate2=0.2,l1=50):
        # We optimize dropout rate in a convolutional neural network.
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.drop1=nn.Dropout2d(p=dropout_rate)  
        
        self.fc1 = nn.Linear(32 * 7 * 7, l1)
        self.drop2=nn.Dropout2d(p=dropout_rate2)
        
        self.fc2 = nn.Linear(l1, 10)

    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x),kernel_size = 2))
        
        x = F.relu(F.max_pool2d(self.conv2(x),kernel_size = 2))
        x = self.drop1(x)
        
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        
        x = self.drop2(x)
        x = self.fc2(x)
        return x

In [512]:
from torch import nn
from skorch import NeuralNetClassifier


y_train = torch.tensor([y for x, y in iter(train_dataset)])
# y_train = y_train.type(torch.LongTensor)
torch.manual_seed(0)

x = torch.tensor(X_train[0:42000].values, dtype=torch.float32)

net = NeuralNetClassifier(
    ConvNet,
    max_epochs=10,
    iterator_train__num_workers=4,
    iterator_valid__num_workers=4,
    lr=1e-3,
    batch_size=1,
    optimizer=optim.Adam,
    # DEVICE=device
)

net.fit(train_dataset, y=y_train)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


RuntimeError: ignored

In [ ]:
len(train_dataloader)

In [ ]:
x = F.normalize(dataloader)